In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [17]:
data_path = "cifar-10-batches-py/data_batch_"
batches = []

for i in range(1, 6):
    batches.append(unpickle(data_path + str(i)))
    
#batch1 = unpickle(data_path)

test_path = "cifar-10-batches-py/test_batch"
test = unpickle(test_path)

In [19]:
batches[1]

{b'batch_label': b'training batch 2 of 5',
 b'data': array([[ 35,  27,  25, ..., 169, 168, 168],
        [ 20,  20,  18, ..., 111,  97,  51],
        [116, 115, 155, ...,  18,  84, 124],
        ..., 
        [127, 139, 155, ..., 197, 192, 191],
        [190, 200, 208, ..., 163, 182, 192],
        [177, 174, 182, ..., 119, 127, 136]], dtype=uint8),
 b'filenames': [b'auto_s_000241.png',
  b'bufo_viridis_s_001109.png',
  b'rana_catesbeiana_s_000111.png',
  b'banana_boat_s_000728.png',
  b'powerboat_s_001487.png',
  b'felis_catus_s_001376.png',
  b'wapiti_s_001127.png',
  b'goliath_frog_s_000230.png',
  b'stealth_fighter_s_000898.png',
  b'bullfrog_s_001623.png',
  b'stealth_fighter_s_001883.png',
  b'mouser_s_001482.png',
  b'bufo_marinus_s_001427.png',
  b'grass_frog_s_000425.png',
  b'puppy_s_002462.png',
  b'cervus_elaphus_s_001192.png',
  b'cargo_ship_s_000773.png',
  b'mouser_s_000261.png',
  b'dunnock_s_000201.png',
  b'western_toad_s_000205.png',
  b'monoplane_s_001332.png',
  b'f

In [36]:
#train_data = batch1[b'data']
#train_labels = batch1[b'labels']
X_train = batches[0][b'data']
y_train = batches[0][b'labels']
for i in range(1,5):
    X_train = np.concatenate((X_train, batches[i][b'data']))
    y_train = np.concatenate((y_train, batches[i][b'labels']))
    
test_data = test[b'data']
test_labels = test[b'labels']

X_test_full = test_data
y_test_full = np.array(test_labels)

In [37]:
X_train.shape

(50000, 3072)

In [44]:
X_train[0].shape

(3072,)

In [38]:
y_train.shape

(50000,)

In [39]:
numbers = np.array(list(range(X_test_full.shape[0])))
indices = np.random.choice(numbers, X_test_full.shape[0]//2, replace=False)

other_indices = [x not in indices for x in numbers]

X_test = X_test_full[indices]
y_test = y_test_full.reshape(-1,)[indices] 

X_val = X_test_full[other_indices]
y_val = y_test_full.reshape(-1,)[other_indices]

In [40]:
height = 32
width = 32
channels = 3
n_inputs = height * width * channels

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 32
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

In [45]:
tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 16 * 16])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [8]:
def fetch_batch(epoch, batch_size):
    np.random.seed(epoch * batch_size) 
    indices = np.random.randint(X_train.shape[0], size=batch_size) 
    X_batch = X_train[indices]
    y_batch = y_train[indices] 
    return X_batch, y_batch

In [46]:
n_epochs = 25
batch_size = 64

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size):
            
            
            #X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch, y_batch = fetch_batch(epoch, batch_size)
            
            
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train accuracy:", acc_train)
        
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Test accuracy:", acc_test)

InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [256,10] and labels shape [64]
	 [[Node: train/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](output/output/BiasAdd, _arg_inputs/y_0_1/_3)]]

Caused by op 'train/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "c:\users\brand\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\brand\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\brand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\brand\appdata\local\programs\python\python36\lib\site-packages\traitlets\co